# Inference and measurement

See <https://www.w3.org/standards/semanticweb/inference>

In [1]:
import kglab

namespaces = {
    "foaf": "http://xmlns.com/foaf/0.1/",
    "gorm": "http://example.org/sagas#",
    "owl": "http://www.w3.org/2002/07/owl#",
    "rel": "http://purl.org/vocab/relationship/",
    }

kg = kglab.KnowledgeGraph(
    name = "KG example for SKOS/OWL inference",
    base_uri = "http://example.org/sagas#",
    namespaces=namespaces
    )

kg.load_ttl("dat/gorm.ttl")

In [2]:
text = kg.save_ttl_text()
print(text)

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix gorm: <http://example.org/sagas#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

gorm:Astrid a gorm:Viking ;
    gorm:childOf gorm:Bodil,
        gorm:Leif .

gorm:childOf rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking ;
    owl:inverseOf gorm:anscestorOf .

gorm:spouseOf a owl:SymmetricProperty ;
    rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking .

gorm:Bjorn a gorm:Viking ;
    gorm:childOf gorm:Gorm .

gorm:Bodil a gorm:Viking ;
    gorm:spouseOf gorm:Leif .

gorm:Gorm a gorm:Viking .

gorm:Leif a gorm:Viking ;
    gorm:childOf gorm:Bjorn .

gorm:Viking a foaf:Person .




In [3]:
import pandas as pd
pd.set_option("max_rows", None)

measure = kglab.Measure()
measure.measure_graph(kg)

print("edges", measure.edge_count)
print("nodes", measure.node_count)

edges 17
nodes 11


In [5]:
sparql = """
SELECT ?elder ?viking
  WHERE {
      ?elder gorm:anscestorOf ?viking
  }
  ORDER BY ASC(?viking)
  """

df = kg.query_as_df(sparql)
df

""


In [6]:
sparql = """
SELECT ?viking1 ?viking2
  WHERE {
      ?viking1 gorm:spouseOf ?viking2
  }
  """

df = kg.query_as_df(sparql)
df

,viking1,viking2
0,http://example.org/sagas#Bodil,http://example.org/sagas#Leif


In [7]:
kg.infer_owlrl_closure()

In [8]:
text = kg.save_ttl_text()
print(text)

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix gorm: <http://example.org/sagas#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

gorm:childOf rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking ;
    owl:inverseOf gorm:anscestorOf ;
    = gorm:childOf .

gorm:spouseOf a owl:SymmetricProperty ;
    rdfs:domain gorm:Viking ;
    rdfs:range gorm:Viking ;
    = gorm:spouseOf .

rdf:HTML a rdfs:Datatype ;
    = rdf:HTML .

rdf:LangString a rdfs:Datatype ;
    = rdf:LangString .

rdf:PlainLiteral a rdfs:Datatype ;
    = rdf:PlainLiteral .

rdf:XMLLiteral a rdfs:Datatype ;
    = rdf:XMLLiteral .

rdf:type = rdf:type .

rdfs:Literal a rdfs:Datatype ;
    = rdfs:Literal .

rdfs:comment a owl:AnnotationProperty ;
    = rdfs:comment .

rdfs:domain = rdfs:domain .

rdfs:isDefinedBy a owl:AnnotationProperty ;
    = rdfs:isD

In [9]:
measure = kglab.Measure()
measure.measure_graph(kg)

print("edges", measure.edge_count)
print("nodes", measure.node_count)

edges 142
nodes 68


In [10]:
sparql = """
SELECT ?elder_viking ?viking
  WHERE {
      ?elder_viking gorm:anscestorOf ?viking
  }
  ORDER BY ASC(?viking)
  """

df = kg.query_as_df(sparql)
df

,elder_viking,viking
0,http://example.org/sagas#Leif,http://example.org/sagas#Astrid
1,http://example.org/sagas#Bodil,http://example.org/sagas#Astrid
2,http://example.org/sagas#Gorm,http://example.org/sagas#Bjorn
3,http://example.org/sagas#Bjorn,http://example.org/sagas#Leif


In [11]:
sparql = """
SELECT ?viking1 ?viking2
  WHERE {
      ?viking1 gorm:spouseOf ?viking2
  }
  """

df = kg.query_as_df(sparql)
df

,viking1,viking2
0,http://example.org/sagas#Leif,http://example.org/sagas#Bodil
1,http://example.org/sagas#Bodil,http://example.org/sagas#Leif


In [12]:
measure.s_gen.get_tally()

,count
http://www.w3.org/2002/07/owl#Nothing,5
http://example.org/sagas#Leif,5
http://example.org/sagas#childOf,4
http://example.org/sagas#Bodil,4
http://example.org/sagas#Astrid,4
http://example.org/sagas#spouseOf,4
http://www.w3.org/2002/07/owl#Thing,4
http://example.org/sagas#Bjorn,4
http://example.org/sagas#Gorm,3
http://www.w3.org/1999/02/22-rdf-syntax-ns#HTML,2


In [13]:
measure.p_gen.get_tally()

,count
http://www.w3.org/2002/07/owl#sameAs,68
http://www.w3.org/1999/02/22-rdf-syntax-ns#type,54
http://example.org/sagas#childOf,4
http://example.org/sagas#anscestorOf,4
http://www.w3.org/2000/01/rdf-schema#subClassOf,3
http://www.w3.org/2000/01/rdf-schema#domain,2
http://www.w3.org/2000/01/rdf-schema#range,2
http://example.org/sagas#spouseOf,2
http://www.w3.org/2002/07/owl#equivalentClass,2
http://www.w3.org/2002/07/owl#inverseOf,1


In [14]:
measure.o_gen.get_tally()

,count
http://www.w3.org/2000/01/rdf-schema#Datatype,37
http://example.org/sagas#Viking,10
http://www.w3.org/2002/07/owl#AnnotationProperty,10
http://www.w3.org/2002/07/owl#Thing,4
http://example.org/sagas#Leif,4
http://example.org/sagas#Astrid,3
http://www.w3.org/2002/07/owl#Nothing,3
http://example.org/sagas#Bodil,3
http://www.w3.org/2002/07/owl#Class,3
http://example.org/sagas#Bjorn,3
